In [1]:
from ast import literal_eval
from os import listdir
from os.path import isfile, join
from scipy.sparse import csr_matrix, load_npz, save_npz
from tqdm import tqdm
from sklearn.preprocessing import normalize
import yaml
import seaborn as sns
import datetime
import json
import numpy as np
import pandas as pd
import time
import yaml
import scipy.sparse as sparse
from ast import literal_eval

# For Python2 this have to be done
# from __future__ import division


### Import Toronto data as df,  userJson as df_user

In [2]:
# read csv data
df_toronto_review = pd.read_csv('../data/toronto_reviews.csv')
df_business = pd.read_csv('../data/businesses_final_toronto.csv')

In [3]:
# rename business id
df_business.rename(columns={'id': 'business_id'}, inplace=True)

In [4]:
df_business.head(2)

,Unnamed: 0,alias,categories,coordinates,display_phone,distance,business_id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,0,the-ossington-toronto,"[{'alias': 'divebars', 'title': 'Dive Bars'}]","{'latitude': 43.6456, 'longitude': -79.41903}",+1 416-850-0161,4831.221092,KUdj08tVC3tvIT0rM6V8uQ,https://s3-media1.fl.yelpcdn.com/bphoto/dEQPAb...,False,"{'address1': '61 Ossington Avenue', 'address2'...",The Ossington,1.416850e+10,$$,3.0,46,[],https://www.yelp.com/biz/the-ossington-toronto...
1,1,life-is-sweet-toronto-3,"[{'alias': 'desserts', 'title': 'Desserts'}, {...","{'latitude': 43.67212500034, 'longitude': -79....",+1 416-698-0555,1716.906703,rnjFHvb1I-KlnGvv_o_Zxg,https://s3-media2.fl.yelpcdn.com/bphoto/s11QxZ...,False,"{'address1': '2345 Queen Street E', 'address2'...",Life is Sweet,1.416698e+10,$$,4.0,10,[],https://www.yelp.com/biz/life-is-sweet-toronto...


In [115]:
print("total business numbers:", len(df_business))

total business numbers: 32123


In [5]:
restaurantString = "restaurant|Bar|Bakeries|food|Coffee|Tea|Waffles|Desserts\
|sandwiches|Cupcakes|cafes|Breakfast|Brunch|Lunch|Dinner|chicken|wings\
|singaporean|malaysian|indonesian|russian|taiwanese|american|chinese|vietnamese|moroccan\
|cantonese|caribbean|mideastern|asian|italian|korean|himalayan|african|nepalese|laotian|sushi|hawaiian\
|mediterranean|caterers|fish|chips|fishnchips|thai|salad|seafood|irish|portuguese|french|vegan|raw_food\
|pizza|latin|lebanese|burgers|gluten-free|japanese|pakistani|mexican|beer|sushi|venues|argentine|lounges|barbeque\
|canadian|dimsum|halal|afghan|donuts|juice|smoothies|bagels|ramen|icecream|pubs|falafel\
|noodles|soup|cheese|colombian|turkish|ethiopian|peruvian|hungarian|ukrainian|indian\
|persian|brazilian|vegetarian|austrian|arabian|haitian|cambodian|german|greek"

# We only take restaurant business
cond_category_restaurant = df_business["categories"].str.contains(restaurantString,case=False)

df_business_Restaurant = df_business[cond_category_restaurant]
print('size: ', len(df_business_Restaurant))

size:  10301


In [6]:
cond_category_Not_restaurant = ~df_business["categories"].str.contains(restaurantString,case=False)
business_not_res = df_business[cond_category_Not_restaurant]

In [118]:
# for index in range(0,100):
#     print(business_not_res['categories'][index])
#business_not_res['categories']

for index, value in business_not_res['categories'].items():
    print("Index : {}, Value : {}".format(index, value))

Index : 3, Value : [{'alias': 'healthtrainers', 'title': 'Trainers'}, {'alias': 'gyms', 'title': 'Gyms'}]
Index : 4, Value : [{'alias': 'locksmiths', 'title': 'Keys & Locksmiths'}]
Index : 6, Value : [{'alias': 'privatetutors', 'title': 'Private Tutors'}]
Index : 7, Value : [{'alias': 'medcenters', 'title': 'Medical Centers'}, {'alias': 'hospitals', 'title': 'Hospitals'}]
Index : 8, Value : [{'alias': 'chiropractors', 'title': 'Chiropractors'}, {'alias': 'massage_therapy', 'title': 'Massage Therapy'}, {'alias': 'naturopathic', 'title': 'Naturopathic/Holistic'}]
Index : 9, Value : [{'alias': 'movers', 'title': 'Movers'}, {'alias': 'selfstorage', 'title': 'Self Storage'}]
Index : 10, Value : [{'alias': 'autorepair', 'title': 'Auto Repair'}, {'alias': 'car_dealers', 'title': 'Car Dealers'}]
Index : 12, Value : [{'alias': 'shopping', 'title': 'Shopping'}, {'alias': 'eventplanning', 'title': 'Party & Event Planning'}]
Index : 13, Value : [{'alias': 'acupuncture', 'title': 'Acupuncture'}, {'

Index : 188, Value : [{'alias': 'dentalhygienists', 'title': 'Dental Hygienists'}, {'alias': 'generaldentistry', 'title': 'General Dentistry'}, {'alias': 'cosmeticdentists', 'title': 'Cosmetic Dentists'}]
Index : 189, Value : [{'alias': 'homecleaning', 'title': 'Home Cleaning'}, {'alias': 'officecleaning', 'title': 'Office Cleaning'}]
Index : 190, Value : [{'alias': 'chiropractors', 'title': 'Chiropractors'}, {'alias': 'acupuncture', 'title': 'Acupuncture'}, {'alias': 'massage', 'title': 'Massage'}]
Index : 192, Value : [{'alias': 'electronics', 'title': 'Electronics'}, {'alias': 'securitysystems', 'title': 'Security Systems'}]
Index : 193, Value : [{'alias': 'securitysystems', 'title': 'Security Systems'}]
Index : 194, Value : [{'alias': 'accountants', 'title': 'Accountants'}]
Index : 195, Value : [{'alias': 'medicalspa', 'title': 'Medical Spas'}, {'alias': 'laser_hair_removal', 'title': 'Laser Hair Removal'}]
Index : 197, Value : [{'alias': 'libraries', 'title': 'Libraries'}]
Index :

Index : 380, Value : [{'alias': 'vintage', 'title': 'Used, Vintage & Consignment'}, {'alias': 'accessories', 'title': 'Accessories'}]
Index : 381, Value : [{'alias': 'autorepair', 'title': 'Auto Repair'}, {'alias': 'tires', 'title': 'Tires'}]
Index : 383, Value : [{'alias': 'othersalons', 'title': 'Nail Salons'}, {'alias': 'eyelashservice', 'title': 'Eyelash Service'}, {'alias': 'waxing', 'title': 'Waxing'}]
Index : 384, Value : [{'alias': 'flooring', 'title': 'Flooring'}, {'alias': 'carpeting', 'title': 'Carpeting'}]
Index : 385, Value : [{'alias': 'apartments', 'title': 'Apartments'}]
Index : 386, Value : [{'alias': 'limos', 'title': 'Limos'}]
Index : 389, Value : [{'alias': 'car_dealers', 'title': 'Car Dealers'}]
Index : 391, Value : [{'alias': 'headshops', 'title': 'Head Shops'}, {'alias': 'accessories', 'title': 'Accessories'}]
Index : 392, Value : [{'alias': 'petstore', 'title': 'Pet Stores'}]
Index : 393, Value : [{'alias': 'waxing', 'title': 'Waxing'}, {'alias': 'threadingservi

Index : 566, Value : [{'alias': 'homecleaning', 'title': 'Home Cleaning'}, {'alias': 'officecleaning', 'title': 'Office Cleaning'}, {'alias': 'interiordesign', 'title': 'Interior Design'}]
Index : 567, Value : [{'alias': 'computers', 'title': 'Computers'}]
Index : 569, Value : [{'alias': 'car_dealers', 'title': 'Car Dealers'}]
Index : 570, Value : [{'alias': 'reflexology', 'title': 'Reflexology'}, {'alias': 'massage_therapy', 'title': 'Massage Therapy'}]
Index : 571, Value : [{'alias': 'martialarts', 'title': 'Martial Arts'}]
Index : 572, Value : [{'alias': 'nonprofit', 'title': 'Community Service/Non-Profit'}]
Index : 573, Value : [{'alias': 'servicestations', 'title': 'Gas Stations'}]
Index : 574, Value : [{'alias': 'locksmiths', 'title': 'Keys & Locksmiths'}, {'alias': 'securitysystems', 'title': 'Security Systems'}, {'alias': 'doorsales', 'title': 'Door Sales/Installation'}]
Index : 580, Value : [{'alias': 'hobbyshops', 'title': 'Hobby Shops'}, {'alias': 'electronics', 'title': 'El

Index : 751, Value : [{'alias': 'kitchenandbath', 'title': 'Kitchen & Bath'}, {'alias': 'cabinetry', 'title': 'Cabinetry'}]
Index : 752, Value : [{'alias': 'officecleaning', 'title': 'Office Cleaning'}]
Index : 753, Value : [{'alias': 'physicaltherapy', 'title': 'Physical Therapy'}, {'alias': 'chiropractors', 'title': 'Chiropractors'}, {'alias': 'acupuncture', 'title': 'Acupuncture'}]
Index : 755, Value : [{'alias': 'doorsales', 'title': 'Door Sales/Installation'}]
Index : 757, Value : [{'alias': 'gyms', 'title': 'Gyms'}, {'alias': 'kickboxing', 'title': 'Kickboxing'}]
Index : 758, Value : [{'alias': 'publictransport', 'title': 'Public Transportation'}]
Index : 759, Value : [{'alias': 'web_design', 'title': 'Web Design'}]
Index : 761, Value : [{'alias': 'hotels', 'title': 'Hotels'}]
Index : 762, Value : [{'alias': 'deptstores', 'title': 'Department Stores'}, {'alias': 'stationery', 'title': 'Cards & Stationery'}, {'alias': 'homedecor', 'title': 'Home Decor'}]
Index : 765, Value : [{'al

...

Index : 27202, Value : [{'alias': 'laundryservices', 'title': 'Laundry Services'}]
Index : 27203, Value : [{'alias': 'autorepair', 'title': 'Auto Repair'}, {'alias': 'bodyshops', 'title': 'Body Shops'}]
Index : 27204, Value : [{'alias': 'hair', 'title': 'Hair Salons'}, {'alias': 'laser_hair_removal', 'title': 'Laser Hair Removal'}, {'alias': 'spas', 'title': 'Day Spas'}]
Index : 27205, Value : [{'alias': 'reupholstery', 'title': 'Furniture Reupholstery'}]
Index : 27206, Value : [{'alias': 'autorepair', 'title': 'Auto Repair'}]
Index : 27210, Value : [{'alias': 'tattoo', 'title': 'Tattoo'}]
Index : 27212, Value : [{'alias': 'gyms', 'title': 'Gyms'}, {'alias': 'yoga', 'title': 'Yoga'}]
Index : 27213, Value : [{'alias': 'autorepair', 'title': 'Auto Repair'}]
Index : 27215, Value : [{'alias': 'othersalons', 'title': 'Nail Salons'}]
Index : 27216, Value : [{'alias': 'galleries', 'title': 'Art Galleries'}]
Index : 27218, Value : [{'alias': 'laundryservices', 'title': 'Laundry Services'}]
Ind

Index : 27459, Value : [{'alias': 'landmarks', 'title': 'Landmarks & Historical Buildings'}]
Index : 27460, Value : [{'alias': 'tours', 'title': 'Tours'}]
Index : 27463, Value : [{'alias': 'womenscloth', 'title': "Women's Clothing"}]
Index : 27464, Value : [{'alias': 'shoes', 'title': 'Shoe Stores'}]
Index : 27466, Value : [{'alias': 'hair', 'title': 'Hair Salons'}]
Index : 27467, Value : [{'alias': 'florists', 'title': 'Florists'}]
Index : 27468, Value : [{'alias': 'optometrists', 'title': 'Optometrists'}, {'alias': 'opticians', 'title': 'Eyewear & Opticians'}]
Index : 27469, Value : [{'alias': 'theater', 'title': 'Performing Arts'}]
Index : 27471, Value : [{'alias': 'chiropractors', 'title': 'Chiropractors'}]
Index : 27475, Value : [{'alias': 'lawyers', 'title': 'Lawyers'}]
Index : 27476, Value : [{'alias': 'cosmetics', 'title': 'Cosmetics & Beauty Supply'}]
Index : 27477, Value : [{'alias': 'currencyexchange', 'title': 'Currency Exchange'}]
Index : 27478, Value : [{'alias': 'jewelry

Index : 27730, Value : [{'alias': 'convenience', 'title': 'Convenience Stores'}]
Index : 27731, Value : [{'alias': 'generaldentistry', 'title': 'General Dentistry'}, {'alias': 'orthodontists', 'title': 'Orthodontists'}, {'alias': 'teethwhitening', 'title': 'Teeth Whitening'}]
Index : 27732, Value : [{'alias': 'apartments', 'title': 'Apartments'}]
Index : 27733, Value : [{'alias': 'carrental', 'title': 'Car Rental'}]
Index : 27736, Value : [{'alias': 'auto_detailing', 'title': 'Auto Detailing'}, {'alias': 'carwash', 'title': 'Car Wash'}]
Index : 27737, Value : [{'alias': 'tires', 'title': 'Tires'}, {'alias': 'autorepair', 'title': 'Auto Repair'}]
Index : 27738, Value : [{'alias': 'limos', 'title': 'Limos'}]
Index : 27739, Value : [{'alias': 'financialservices', 'title': 'Financial Services'}]
Index : 27741, Value : [{'alias': 'tours', 'title': 'Tours'}]
Index : 27746, Value : [{'alias': 'petstore', 'title': 'Pet Stores'}]
Index : 27747, Value : [{'alias': 'itservices', 'title': 'IT Serv

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




Index : 31613, Value : [{'alias': 'intlgrocery', 'title': 'International Grocery'}]
Index : 31619, Value : [{'alias': 'dancestudio', 'title': 'Dance Studios'}]
Index : 31620, Value : [{'alias': 'immigrationlawyers', 'title': 'Immigration Law'}]
Index : 31624, Value : [{'alias': 'divorce', 'title': 'Divorce & Family Law'}, {'alias': 'estateplanning', 'title': 'Estate Planning Law'}]
Index : 31628, Value : [{'alias': 'hair', 'title': 'Hair Salons'}]
Index : 31629, Value : [{'alias': 'insurance', 'title': 'Insurance'}]
Index : 31631, Value : [{'alias': 'insurance', 'title': 'Insurance'}]
Index : 31633, Value : [{'alias': 'insurance', 'title': 'Insurance'}]
Index : 31634, Value : [{'alias': 'itservices', 'title': 'IT Services & Computer Repair'}]
Index : 31636, Value : [{'alias': 'insurance', 'title': 'Insurance'}]
Index : 31638, Value : [{'alias': 'graphicdesign', 'title': 'Graphic Design'}]
Index : 31639, Value : [{'alias': 'healthmarkets', 'title': 'Health Markets'}]
Index : 31649, Val

In [119]:
#Checking business name
df_business['name'][31600]

'Souvlaki Kalamaki'

In [7]:
# Drop duplicate values in df_business
df_business_clean = df_business_Restaurant.drop_duplicates(subset ="business_id", 
                     keep = False)

In [8]:
# Rename Month as Day and Day as Month(error occurs on original data)
df_toronto_review=df_toronto_review.rename(columns = {'Month':'new_Day'})
df_toronto_review=df_toronto_review.rename(columns = {'Day':'Month'})
df_toronto_review=df_toronto_review.rename(columns = {'new_Day':'Day'})

In [33]:
# df_toronto_review.shape

In [34]:
# df_business_clean.shape

In [9]:
# inner join two full dataset
df = pd.merge(df_toronto_review, df_business_clean, on='business_id', how='inner')

In [36]:
df.shape

(510285, 39)

In [10]:
# merge the date and turn it into string
df['date'] = pd.to_datetime(df[['Year', 'Month','Day']])
df['date'] = df['date'].astype(str)

In [11]:
# rename the columns to match the algorithm 
df.rename(columns={'rating_x': 'stars','rating_y': 'business_stars','review_text': 'text',}, inplace=True)

In [12]:
# get rid of the rows without user_id
df = df[~df["user_id"].isnull()]

In [13]:
pd.set_option('display.max_columns', None)
df

,Unnamed: 0_x,Unnamed: 0.1,business_id,friend_count,ghost,img_dsc,img_url,nr,photo_count,stars,review_count_x,review_date,review_id,review_language,text,ufc,user_id,user_loc,vote_count,Updated,Year,Day,Month,Unnamed: 0_y,alias,categories,coordinates,display_phone,distance,image_url,is_closed,location,name,phone,price,business_stars,review_count_y,transactions,url,date
0,6,6,Xo1LNzhnwE-ilqsM3ybs9Q,105,False,"['Photo of Happy Lemon - Markham, ON, Canada. ...",['https://s3-media4.fl.yelpcdn.com/bphoto/tu7j...,False,325.0,4.0,122,"['8', '23', '2016']",rqjzFEb1rQ09CgDVlGxZug,en,I ordered the lemon mango slush and the lemon ...,"[1, 1, 1]",zsJFjhBQEFQ6gJ7BsNM_Ug,"Toronto, Canada",1.0,False,2016,23,8,21653,happy-lemon-markham,"[{'alias': 'bubbletea', 'title': 'Bubble Tea'}...","{'latitude': 43.8258200636421, 'longitude': -7...",+1 416-855-0768,2867.240253,https://s3-media4.fl.yelpcdn.com/bphoto/ATj6Mv...,False,"{'address1': 'B25 4300 Steeles Avenue E', 'add...",Happy Lemon,1.416855e+10,$,3.5,32,[],https://www.yelp.com/biz/happy-lemon-markham?a...,2016-08-23
1,7,7,Xo1LNzhnwE-ilqsM3ybs9Q,171,False,"['Photo of Happy Lemon - Markham, ON, Canada. ...",['https://s3-media3.fl.yelpcdn.com/bphoto/h110...,False,307.0,3.0,114,"['10', '2', '2016']",P7S5VclOJEihkqpA7LscPw,en,"Came here on a Sunday afternoon, it wasn't bus...","[1, 0, 0]",P7YuMh74-I2cDq7oU8frww,"York Regional Municipality, Canada",1.0,False,2016,2,10,21653,happy-lemon-markham,"[{'alias': 'bubbletea', 'title': 'Bubble Tea'}...","{'latitude': 43.8258200636421, 'longitude': -7...",+1 416-855-0768,2867.240253,https://s3-media4.fl.yelpcdn.com/bphoto/ATj6Mv...,False,"{'address1': 'B25 4300 Steeles Avenue E', 'add...",Happy Lemon,1.416855e+10,$,3.5,32,[],https://www.yelp.com/biz/happy-lemon-markham?a...,2016-10-02
2,8,8,Xo1LNzhnwE-ilqsM3ybs9Q,239,False,"['Photo of Happy Lemon - Markham, ON, Canada. ...",['https://s3-media4.fl.yelpcdn.com/bphoto/tS6Y...,False,78.0,5.0,49,"['11', '6', '2016']",oHatxr1JfRewl0g49--C7g,en,{Grapefruit & Yakult Green Tea with Aloe Jelly...,"[0, 0, 0]",C91it8b3odRg0503asUaJA,"Markham, Canada",0.0,False,2016,6,11,21653,happy-lemon-markham,"[{'alias': 'bubbletea', 'title': 'Bubble Tea'}...","{'latitude': 43.8258200636421, 'longitude': -7...",+1 416-855-0768,2867.240253,https://s3-media4.fl.yelpcdn.com/bphoto/ATj6Mv...,False,"{'address1': 'B25 4300 Steeles Avenue E', 'add...",Happy Lemon,1.416855e+10,$,3.5,32,[],https://www.yelp.com/biz/happy-lemon-markham?a...,2016-11-06
3,9,9,Xo1LNzhnwE-ilqsM3ybs9Q,10,False,"['Photo of Happy Lemon - Markham, ON, Canada',...",['https://s3-media3.fl.yelpcdn.com/bphoto/rfB0...,False,504.0,4.0,164,"['9', '25', '2016']",4bnmXCDq3T7kLtTzesT_Cw,en,Saw this newly opened bubble tea shop and want...,"[0, 0, 0]",dAB-v4B-5_8QgTcoVSnZOw,"Vaughan, Canada",0.0,False,2016,25,9,21653,happy-lemon-markham,"[{'alias': 'bubbletea', 'title': 'Bubble Tea'}...","{'latitude': 43.8258200636421, 'longitude': -7...",+1 416-855-0768,2867.240253,https://s3-media4.fl.yelpcdn.com/bphoto/ATj6Mv...,False,"{'address1': 'B25 4300 Steeles Avenue E', 'add...",Happy Lemon,1.416855e+10,$,3.5,32,[],https://www.yelp.com/biz/happy-lemon-markham?a...,2016-09-25
4,10,10,Xo1LNzhnwE-ilqsM3ybs9Q,121,False,[],[],False,39.0,5.0,27,"['9', '6', '2016']",WgSTb6L_OMcTeUq5vKYmxA,en,Walked by pmall and the poster of the salty ch...,"[2, 2, 1]",UxwiJpYSqOZJQzUMF6Xl6A,"Toronto, Canada",3.0,False,2016,6,9,21653,happy-lemon-markham,"[{'alias': 'bubbletea', 'title': 'Bubble Tea'}...","{'latitude': 43.8258200636421, 'longitude': -7...",+1 416-855-0768,2867.240253,https://s3-media4.fl.yelpcdn.com/bphoto/ATj6Mv...,False,"{'address1': 'B25 4300 Steeles Avenue E', 'add...",Happy Lemon,1.416855e+10,$,3.5,32,[],https://www.yelp.com/biz/happy-lemon-markham?a...,2016-09-06
5,11,11,Xo1LNzhnwE-ilqsM3ybs9Q,139,False,"['Photo of Happy Lemon - Markham, ON, Canada. ...",['https://s3-media3.fl.yelpcdn.com/bphoto/5GBq...,False,181.0,3.0,81,"['2', '21', '2017']",bZzY0Oxo0tqfDgWC20tRCA,en,I got the mango mat

In [41]:
len(df)

413614

In [42]:
df

,Unnamed: 0_x,Unnamed: 0.1,business_id,friend_count,ghost,img_dsc,img_url,nr,photo_count,stars,review_count_x,review_date,review_id,review_language,text,ufc,user_id,user_loc,vote_count,Updated,Year,Day,Month,Unnamed: 0_y,alias,categories,coordinates,display_phone,distance,image_url,is_closed,location,name,phone,price,business_stars,review_count_y,transactions,url,date
0,6,6,Xo1LNzhnwE-ilqsM3ybs9Q,105,False,"['Photo of Happy Lemon - Markham, ON, Canada. ...",['https://s3-media4.fl.yelpcdn.com/bphoto/tu7j...,False,325.0,4.0,122,"['8', '23', '2016']",rqjzFEb1rQ09CgDVlGxZug,en,I ordered the lemon mango slush and the lemon ...,"[1, 1, 1]",zsJFjhBQEFQ6gJ7BsNM_Ug,"Toronto, Canada",1.0,False,2016,23,8,21653,happy-lemon-markham,"[{'alias': 'bubbletea', 'title': 'Bubble Tea'}...","{'latitude': 43.8258200636421, 'longitude': -7...",+1 416-855-0768,2867.240253,https://s3-media4.fl.yelpcdn.com/bphoto/ATj6Mv...,False,"{'address1': 'B25 4300 Steeles Avenue E', 'add...",Happy Lemon,1.416855e+10,$,3.5,32,[],https://www.yelp.com/biz/happy-lemon-markham?a...,2016-08-23
1,7,7,Xo1LNzhnwE-ilqsM3ybs9Q,171,False,"['Photo of Happy Lemon - Markham, ON, Canada. ...",['https://s3-media3.fl.yelpcdn.com/bphoto/h110...,False,307.0,3.0,114,"['10', '2', '2016']",P7S5VclOJEihkqpA7LscPw,en,"Came here on a Sunday afternoon, it wasn't bus...","[1, 0, 0]",P7YuMh74-I2cDq7oU8frww,"York Regional Municipality, Canada",1.0,False,2016,2,10,21653,happy-lemon-markham,"[{'alias': 'bubbletea', 'title': 'Bubble Tea'}...","{'latitude': 43.8258200636421, 'longitude': -7...",+1 416-855-0768,2867.240253,https://s3-media4.fl.yelpcdn.com/bphoto/ATj6Mv...,False,"{'address1': 'B25 4300 Steeles Avenue E', 'add...",Happy Lemon,1.416855e+10,$,3.5,32,[],https://www.yelp.com/biz/happy-lemon-markham?a...,2016-10-02
2,8,8,Xo1LNzhnwE-ilqsM3ybs9Q,239,False,"['Photo of Happy Lemon - Markham, ON, Canada. ...",['https://s3-media4.fl.yelpcdn.com/bphoto/tS6Y...,False,78.0,5.0,49,"['11', '6', '2016']",oHatxr1JfRewl0g49--C7g,en,{Grapefruit & Yakult Green Tea with Aloe Jelly...,"[0, 0, 0]",C91it8b3odRg0503asUaJA,"Markham, Canada",0.0,False,2016,6,11,21653,happy-lemon-markham,"[{'alias': 'bubbletea', 'title': 'Bubble Tea'}...","{'latitude': 43.8258200636421, 'longitude': -7...",+1 416-855-0768,2867.240253,https://s3-media4.fl.yelpcdn.com/bphoto/ATj6Mv...,False,"{'address1': 'B25 4300 Steeles Avenue E', 'add...",Happy Lemon,1.416855e+10,$,3.5,32,[],https://www.yelp.com/biz/happy-lemon-markham?a...,2016-11-06
3,9,9,Xo1LNzhnwE-ilqsM3ybs9Q,10,False,"['Photo of Happy Lemon - Markham, ON, Canada',...",['https://s3-media3.fl.yelpcdn.com/bphoto/rfB0...,False,504.0,4.0,164,"['9', '25', '2016']",4bnmXCDq3T7kLtTzesT_Cw,en,Saw this newly opened bubble tea shop and want...,"[0, 0, 0]",dAB-v4B-5_8QgTcoVSnZOw,"Vaughan, Canada",0.0,False,2016,25,9,21653,happy-lemon-markham,"[{'alias': 'bubbletea', 'title': 'Bubble Tea'}...","{'latitude': 43.8258200636421, 'longitude': -7...",+1 416-855-0768,2867.240253,https://s3-media4.fl.yelpcdn.com/bphoto/ATj6Mv...,False,"{'address1': 'B25 4300 Steeles Avenue E', 'add...",Happy Lemon,1.416855e+10,$,3.5,32,[],https://www.yelp.com/biz/happy-lemon-markham?a...,2016-09-25
4,10,10,Xo1LNzhnwE-ilqsM3ybs9Q,121,False,[],[],False,39.0,5.0,27,"['9', '6', '2016']",WgSTb6L_OMcTeUq5vKYmxA,en,Walked by pmall and the poster of the salty ch...,"[2, 2, 1]",UxwiJpYSqOZJQzUMF6Xl6A,"Toronto, Canada",3.0,False,2016,6,9,21653,happy-lemon-markham,"[{'alias': 'bubbletea', 'title': 'Bubble Tea'}...","{'latitude': 43.8258200636421, 'longitude': -7...",+1 416-855-0768,2867.240253,https://s3-media4.fl.yelpcdn.com/bphoto/ATj6Mv...,False,"{'address1': 'B25 4300 Steeles Avenue E', 'add...",Happy Lemon,1.416855e+10,$,3.5,32,[],https://www.yelp.com/biz/happy-lemon-markham?a...,2016-09-06
5,11,11,Xo1LNzhnwE-ilqsM3ybs9Q,139,False,"['Photo of Happy Lemon - Markham, ON, Canada. ...",['https://s3-media3.fl.yelpcdn.com/bphoto/5GBq...,False,181.0,3.0,81,"['2', '21', '2017']",bZzY0Oxo0tqfDgWC20tRCA,en,I got the mango mat

In [14]:
dfff = df.copy()

change category format

In [15]:
df = dfff.copy()

In [16]:
df.reset_index(inplace = True)

In [17]:
ans = []
for cat in tqdm(df.categories.values):
    temp = yaml.safe_load(cat)
#     ans.append([temp[i]["title"] for i in range(len(temp))])
    cat_str = ""
    for i in range(len(temp)):
        if i == 0:
            cat_str += temp[i]["title"]
            if len(temp) >1:
                cat_str += ", "
        elif i == len(temp)-1:
            cat_str += temp[i]["title"]
        else:
            cat_str += temp[i]["title"]
            cat_str += ", "
    ans.append(cat_str)
    

100%|████████████████████████████████████████████████████████████████████████| 413614/413614 [02:27<00:00, 2806.96it/s]


In [18]:
arr_temp = np.asarray(ans)

In [19]:
df_cat = pd.DataFrame(arr_temp)
df_cat.rename(columns={0: "cat"}, inplace = True)
df_cat

,cat
0,"Bubble Tea, Coffee & Tea"
1,"Bubble Tea, Coffee & Tea"
2,"Bubble Tea, Coffee & Tea"
3,"Bubble Tea, Coffee & Tea"
4,"Bubble Tea, Coffee & Tea"
5,"Bubble Tea, Coffee & Tea"
6,"Bubble Tea, Coffee & Tea"
7,"Bubble Tea, Coffee & Tea"
8,"Bubble Tea, Coffee & Tea"
9,"Bubble Tea, Coffee & Tea"


In [20]:
df["cat"] = df_cat

In [66]:
pd.set_option('display.max_rows', 100000)

In [21]:
del df['categories']

In [22]:
df.rename(columns={"cat": "categories"}, inplace = True)

In [23]:
df

,index,Unnamed: 0_x,Unnamed: 0.1,business_id,friend_count,ghost,img_dsc,img_url,nr,photo_count,stars,review_count_x,review_date,review_id,review_language,text,ufc,user_id,user_loc,vote_count,Updated,Year,Day,Month,Unnamed: 0_y,alias,coordinates,display_phone,distance,image_url,is_closed,location,name,phone,price,business_stars,review_count_y,transactions,url,date,categories
0,0,6,6,Xo1LNzhnwE-ilqsM3ybs9Q,105,False,"['Photo of Happy Lemon - Markham, ON, Canada. ...",['https://s3-media4.fl.yelpcdn.com/bphoto/tu7j...,False,325.0,4.0,122,"['8', '23', '2016']",rqjzFEb1rQ09CgDVlGxZug,en,I ordered the lemon mango slush and the lemon ...,"[1, 1, 1]",zsJFjhBQEFQ6gJ7BsNM_Ug,"Toronto, Canada",1.0,False,2016,23,8,21653,happy-lemon-markham,"{'latitude': 43.8258200636421, 'longitude': -7...",+1 416-855-0768,2867.240253,https://s3-media4.fl.yelpcdn.com/bphoto/ATj6Mv...,False,"{'address1': 'B25 4300 Steeles Avenue E', 'add...",Happy Lemon,1.416855e+10,$,3.5,32,[],https://www.yelp.com/biz/happy-lemon-markham?a...,2016-08-23,"Bubble Tea, Coffee & Tea"
1,1,7,7,Xo1LNzhnwE-ilqsM3ybs9Q,171,False,"['Photo of Happy Lemon - Markham, ON, Canada. ...",['https://s3-media3.fl.yelpcdn.com/bphoto/h110...,False,307.0,3.0,114,"['10', '2', '2016']",P7S5VclOJEihkqpA7LscPw,en,"Came here on a Sunday afternoon, it wasn't bus...","[1, 0, 0]",P7YuMh74-I2cDq7oU8frww,"York Regional Municipality, Canada",1.0,False,2016,2,10,21653,happy-lemon-markham,"{'latitude': 43.8258200636421, 'longitude': -7...",+1 416-855-0768,2867.240253,https://s3-media4.fl.yelpcdn.com/bphoto/ATj6Mv...,False,"{'address1': 'B25 4300 Steeles Avenue E', 'add...",Happy Lemon,1.416855e+10,$,3.5,32,[],https://www.yelp.com/biz/happy-lemon-markham?a...,2016-10-02,"Bubble Tea, Coffee & Tea"
2,2,8,8,Xo1LNzhnwE-ilqsM3ybs9Q,239,False,"['Photo of Happy Lemon - Markham, ON, Canada. ...",['https://s3-media4.fl.yelpcdn.com/bphoto/tS6Y...,False,78.0,5.0,49,"['11', '6', '2016']",oHatxr1JfRewl0g49--C7g,en,{Grapefruit & Yakult Green Tea with Aloe Jelly...,"[0, 0, 0]",C91it8b3odRg0503asUaJA,"Markham, Canada",0.0,False,2016,6,11,21653,happy-lemon-markham,"{'latitude': 43.8258200636421, 'longitude': -7...",+1 416-855-0768,2867.240253,https://s3-media4.fl.yelpcdn.com/bphoto/ATj6Mv...,False,"{'address1': 'B25 4300 Steeles Avenue E', 'add...",Happy Lemon,1.416855e+10,$,3.5,32,[],https://www.yelp.com/biz/happy-lemon-markham?a...,2016-11-06,"Bubble Tea, Coffee & Tea"
3,3,9,9,Xo1LNzhnwE-ilqsM3ybs9Q,10,False,"['Photo of Happy Lemon - Markham, ON, Canada',...",['https://s3-media3.fl.yelpcdn.com/bphoto/rfB0...,False,504.0,4.0,164,"['9', '25', '2016']",4bnmXCDq3T7kLtTzesT_Cw,en,Saw this newly opened bubble tea shop and want...,"[0, 0, 0]",dAB-v4B-5_8QgTcoVSnZOw,"Vaughan, Canada",0.0,False,2016,25,9,21653,happy-lemon-markham,"{'latitude': 43.8258200636421, 'longitude': -7...",+1 416-855-0768,2867.240253,https://s3-media4.fl.yelpcdn.com/bphoto/ATj6Mv...,False,"{'address1': 'B25 4300 Steeles Avenue E', 'add...",Happy Lemon,1.416855e+10,$,3.5,32,[],https://www.yelp.com/biz/happy-lemon-markham?a...,2016-09-25,"Bubble Tea, Coffee & Tea"
4,4,10,10,Xo1LNzhnwE-ilqsM3ybs9Q,121,False,[],[],False,39.0,5.0,27,"['9', '6', '2016']",WgSTb6L_OMcTeUq5vKYmxA,en,Walked by pmall and the poster of the salty ch...,"[2, 2, 1]",UxwiJpYSqOZJQzUMF6Xl6A,"Toronto, Canada",3.0,False,2016,6,9,21653,happy-lemon-markham,"{'latitude': 43.8258200636421, 'longitude': -7...",+1 416-855-0768,2867.240253,https://s3-media4.fl.yelpcdn.com/bphoto/ATj6Mv...,False,"{'address1': 'B25 4300 Steeles Avenue E', 'add...",Happy Lemon,1.416855e+10,$,3.5,32,[],https://www.yelp.com/biz/happy-lemon-markham?a...,2016-09-06,"Bubble Tea, Coffee & Tea"
5,5,11,11,Xo1LNzhnwE-ilqsM3ybs9Q,139,False,"['Photo of Happy Lemon - Markham, ON, Canada. ...",['https://s3-media3.fl.yelpcdn.com/bphoto/5GBq...,False,181.0,3.0,81,"['2', '21', '2017']",bZzY0Oxo0tqfDgWC20tRCA,en,I got the mango matcha latte to share with my ...,"[0, 0, 0]",3AmCnDklEKOO3ERIrGFB4Q,"Daly City, CA",0.0,False,2017,21,2,21653

### Data Processing

In [24]:
Export = df.to_json(r'..\\data\\Export_TorontoData.json', orient='records')